In [1]:
%pip install accelerate -U
%pip install transformers
%pip install datasets
%pip install seqeval

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import torch
import transformers
from transformers import AutoTokenizer
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
from transformers import DataCollatorForTokenClassification
from datasets import load_metric
from datasets import Dataset
import numpy as np
import pandas as pd
from huggingface_hub import notebook_login

This next part is annoying but necessary to run the code. You have to have a hugging face account and then go [here](https://huggingface.co/settings/tokens), to get a token.

In [2]:
notebook_login()

In [3]:
label_all_tokens = True # dw about it

# Loading the data

In [4]:
def read_iob2_file(path):
    data = []
    current_words = []
    current_tags = []

    for line in open(path, encoding='utf-8'):
        line = line.strip()

        if line:
            if line[0] == '#':
                continue
            tok = line.split('\t')

            current_words.append(tok[1])
            current_tags.append(tok[2])
        else:
            if current_words:
                data.append((current_words, current_tags))
            current_words = []
            current_tags = []

    if current_tags != []:
        data.append((current_words, current_tags))

    df = pd.DataFrame(data, columns=['words', 'tags'])
    df['id'] = df.index
    df = df[['id', 'words', 'tags']]
    
    return df

In [20]:
train_data = read_iob2_file("en_ewt-ud-train.iob2")
dev_data = read_iob2_file("en_ewt-ud-dev.iob2")
test_data = read_iob2_file("en_ewt-ud-test-masked.iob2")
print(train_data)


          id                                              words  \
0          0             [Where, in, the, world, is, Iguazu, ?]   
1          1                                    [Iguazu, Falls]   
2          2  [Widely, considered, to, be, one, of, the, mos...   
3          3  [The, centre, of, the, falls, is, known, local...   
4          4  [With, the, sun, shinning, down, on, the, spra...   
...      ...                                                ...   
12538  12538  [Even, a, little, time, spent, receiving, whol...   
12539  12539  [S., and, I, have, an, acquaintance, who, has,...   
12540  12540  [The, urge, to, protect, and, gather, them, al...   
12541  12541              [Take, care, ,, my, friend, ,, Linda]   
12542  12542  [I, 'm, sorry, to, say, Elena, 's, story, has,...   

                                                    tags  
0                              [O, O, O, O, O, B-LOC, O]  
1                                         [B-LOC, I-LOC]  
2      [O, O, O, O

## Tag/index dictionary

In [6]:
class Vocab():
    def __init__(self, pad_unk='<PAD>'):
        self.pad_unk = pad_unk
        self.word2idx = {}
        self.idx2word = []

    def getIdx(self, word, add=False):
        if word is None or word == self.pad_unk:
            return None
        if word not in self.word2idx:
            if add:
                idx = len(self.idx2word)
                self.word2idx[word] = idx
                self.idx2word.append(word)
                return idx
            else:
                return None
        return self.word2idx[word]

    def getWord(self, idx):
        return self.idx2word[idx]

label_indices = Vocab()
tags_column = train_data["tags"]

for tags in tags_column:
    for tag in tags:
        label_indices.getIdx(tag, add=True)

print(label_indices.word2idx)

{'O': 0, 'B-LOC': 1, 'I-LOC': 2, 'B-PER': 3, 'B-ORG': 4, 'I-ORG': 5, 'I-PER': 6}


Here I add new columns to the data that have the translated tags

In [21]:
train_data['tag_idx'] = train_data['tags'].apply(lambda x: [label_indices.word2idx[tag] for tag in x])
dev_data['tag_idx'] = dev_data['tags'].apply(lambda x: [label_indices.word2idx[tag] for tag in x])
test_data['tag_idx'] = test_data['tags'].apply(lambda x: [label_indices.word2idx[tag] for tag in x])

model_checkpoint = "distilbert/distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

train_data

,id,words,tags,tag_idx
0,0,"[Where, in, the, world, is, Iguazu, ?]","[O, O, O, O, O, B-LOC, O]","[0, 0, 0, 0, 0, 1, 0]"
1,1,"[Iguazu, Falls]","[B-LOC, I-LOC]","[1, 2]"
2,2,"[Widely, considered, to, be, one, of, the, mos...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,3,"[The, centre, of, the, falls, is, known, local...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,4,"[With, the, sun, shinning, down, on, the, spra...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...
12538,12538,"[Even, a, little, time, spent, receiving, whol...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
12539,12539,"[S., and, I, have, an, acquaintance, who, has,...","[B-PER, O, O, O, O, O, O, O, O, O, O, O, O, O,...","[3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
12540,12540,"[The, urge, to, protect, and, gather, them, al...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
12541,12541,"[Take, care, ,, my, friend, ,, Linda]","[O, O, O, O, O, O, B-PER]","[0, 0, 0, 0, 0, 0, 3]"


## Tokenization

In [11]:
def tokenize_and_align_labels(dataset, word_column, tag_column, tokenizer):
    tokenized_inputs = tokenizer(dataset[word_column].tolist(), truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(dataset[tag_column]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs.data

In [22]:
train_data

,id,words,tags,tag_idx
0,0,"[Where, in, the, world, is, Iguazu, ?]","[O, O, O, O, O, B-LOC, O]","[0, 0, 0, 0, 0, 1, 0]"
1,1,"[Iguazu, Falls]","[B-LOC, I-LOC]","[1, 2]"
2,2,"[Widely, considered, to, be, one, of, the, mos...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,3,"[The, centre, of, the, falls, is, known, local...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,4,"[With, the, sun, shinning, down, on, the, spra...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...
12538,12538,"[Even, a, little, time, spent, receiving, whol...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
12539,12539,"[S., and, I, have, an, acquaintance, who, has,...","[B-PER, O, O, O, O, O, O, O, O, O, O, O, O, O,...","[3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
12540,12540,"[The, urge, to, protect, and, gather, them, al...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
12541,12541,"[Take, care, ,, my, friend, ,, Linda]","[O, O, O, O, O, O, B-PER]","[0, 0, 0, 0, 0, 0, 3]"


In [23]:
test_data

,id,words,tags,tag_idx
0,0,"[What, is, this, Miramar, ?]","[O, O, O, O, O]","[0, 0, 0, 0, 0]"
1,1,"[It, is, a, place, in, Argentina, lol]","[O, O, O, O, O, O, O]","[0, 0, 0, 0, 0, 0, 0]"
2,2,"[what, is, a, good, slogan, for, an, Argentini...","[O, O, O, O, O, O, O, O, O, O]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
3,3,"["", In, Argentina, ,, beef, is, revered, ,, re...","[O, O, O, O, O, O, O, O, O, O, O, O, O]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
4,4,"[Come, see, how, we, continue, this, tradition...","[O, O, O, O, O, O, O, O, O]","[0, 0, 0, 0, 0, 0, 0, 0, 0]"
...,...,...,...,...
2072,2072,"[The, radical, Shiite, Sadr, movement, issued,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2073,2073,"[The, statement, said, ,, "", We, direct, an, a...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2074,2074,"[It, called, a, "", ploy, "", the, assertaion, t...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2075,2075,"[The, convergence, of, views, among, the, more...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [24]:
tokenized_data = tokenize_and_align_labels(train_data, "words", "tag_idx", tokenizer)
tokenized_dev_data = tokenize_and_align_labels(dev_data, "words", "tag_idx", tokenizer)
tokenized_test_data = tokenize_and_align_labels(test_data, "words", "tag_idx", tokenizer)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


# Model implementation

In [13]:
label_list = label_indices.idx2word
batch_size = 16

assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast) # verifies the tokenizers compatibility with hugging face

model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))
model_name = model_checkpoint.split("/")[-1]

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
args = TrainingArguments(
    model_name,
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=True,
)

data_collator = DataCollatorForTokenClassification(tokenizer)
metric = load_metric("seqeval")

C:\Users\miran\AppData\Local\Temp\ipykernel_7868\3675664793.py:13: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval")


c:\Users\miran\AppData\Local\Programs\Python\Python311\Lib\site-packages\datasets\load.py:756: FutureWarning: The repository for seqeval contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/seqeval/seqeval.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [25]:
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

## Training

In [26]:
# turning the data into datasetdicts, to make them compatible with the trainer (otherwise they can't be indexed)
train_dataset = Dataset.from_dict({
    'id': range(len(tokenized_data['input_ids'])),
    'input_ids': tokenized_data['input_ids'],
    'attention_mask': tokenized_data['attention_mask'],
    'labels': tokenized_data['labels']
})

dev_dataset = Dataset.from_dict({
    'id': range(len(tokenized_dev_data['input_ids'])),
    'input_ids': tokenized_dev_data['input_ids'],
    'attention_mask': tokenized_dev_data['attention_mask'],
    'labels': tokenized_dev_data['labels']
})

test_dataset = Dataset.from_dict({
    'id': range(len(tokenized_test_data['input_ids'])),
    'input_ids': tokenized_test_data['input_ids'],
    'attention_mask': tokenized_test_data['attention_mask'],
    'labels': tokenized_test_data['labels']
})

In [35]:
train_dataset_sm = train_dataset.select(range(50))

In [36]:
trainer = Trainer(
    model,
    args,
    train_dataset=train_dataset_sm,
    eval_dataset=dev_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

c:\Users\miran\AppData\Local\Programs\Python\Python311\Lib\site-packages\accelerate\accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [37]:
trainer.train()

  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/126 [00:00<?, ?it/s]

{'eval_loss': 1.5940914154052734, 'eval_precision': 0.008826583592938734, 'eval_recall': 0.024691358024691357, 'eval_f1': 0.013004398546567222, 'eval_accuracy': 0.7966813274690124, 'eval_runtime': 157.8503, 'eval_samples_per_second': 12.677, 'eval_steps_per_second': 0.798, 'epoch': 1.0}


  0%|          | 0/126 [00:00<?, ?it/s]

{'eval_loss': 1.3234262466430664, 'eval_precision': 0.0045045045045045045, 'eval_recall': 0.0007262164124909223, 'eval_f1': 0.0012507817385866164, 'eval_accuracy': 0.9223310675729708, 'eval_runtime': 203.3612, 'eval_samples_per_second': 9.84, 'eval_steps_per_second': 0.62, 'epoch': 2.0}


  0%|          | 0/126 [00:00<?, ?it/s]

{'eval_loss': 1.2209219932556152, 'eval_precision': 0.01282051282051282, 'eval_recall': 0.0007262164124909223, 'eval_f1': 0.0013745704467353953, 'eval_accuracy': 0.9265627082500333, 'eval_runtime': 99.2466, 'eval_samples_per_second': 20.162, 'eval_steps_per_second': 1.27, 'epoch': 3.0}
{'train_runtime': 504.8118, 'train_samples_per_second': 0.297, 'train_steps_per_second': 0.024, 'train_loss': 1.4458163579305012, 'epoch': 3.0}


TrainOutput(global_step=12, training_loss=1.4458163579305012, metrics={'train_runtime': 504.8118, 'train_samples_per_second': 0.297, 'train_steps_per_second': 0.024, 'train_loss': 1.4458163579305012, 'epoch': 3.0})

In [ ]:
#trainer.save_model()

## Evaluation

In [38]:
trainer.evaluate()

  0%|          | 0/126 [00:00<?, ?it/s]

{'eval_loss': 1.2209219932556152,
 'eval_precision': 0.01282051282051282,
 'eval_recall': 0.0007262164124909223,
 'eval_f1': 0.0013745704467353953,
 'eval_accuracy': 0.9265627082500333,
 'eval_runtime': 112.0727,
 'eval_samples_per_second': 17.854,
 'eval_steps_per_second': 1.124,
 'epoch': 3.0}

In [39]:
# evaluating using dev data
dev_dataset_new = Dataset.from_dict({
    'input_ids': dev_dataset['input_ids'],
    'attention_mask': dev_dataset['attention_mask'],
    'labels': dev_dataset['labels']
})

test_dataset_new = Dataset.from_dict({
    'input_ids': test_dataset['input_ids'],
    'attention_mask': test_dataset['attention_mask'],
    'labels': test_dataset['labels']
})

predictions, labels, _ = trainer.predict(test_dataset_new)
predictions = np.argmax(predictions, axis=2)

true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

  0%|          | 0/130 [00:00<?, ?it/s]

c:\Users\miran\AppData\Local\Programs\Python\Python311\Lib\site-packages\seqeval\metrics\v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\miran\AppData\Local\Programs\Python\Python311\Lib\site-packages\seqeval\metrics\v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'LOC': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 0},
 'ORG': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 0},
 'PER': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 0},
 'overall_precision': 0.0,
 'overall_recall': 0.0,
 'overall_f1': 0.0,
 'overall_accuracy': 0.9970841660387249}

In [ ]:
# Print predictions and real values side by side
for i in range(len(true_predictions)):
    print("Example", i+1)
    print("Predicted:", true_predictions[i])
    print("Real:", true_labels[i])
    print()

# Getting the correct format